In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow.keras as keras
import os

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.preprocessing import LabelBinarizer

# 调整TensorFlow的GPU内存分配为按需分配,否则会一次性占满所有内存
tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)

for dirname, _, filenames in os.walk('ges_mnist/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
train_df = pd.read_csv("ges_mnist/sign_mnist_train.csv")
test_df = pd.read_csv("ges_mnist/sign_mnist_test.csv")
# train_df.head()
y_train = train_df['label']
y_test = test_df['label']

del train_df['label']
del test_df['label']

x_train = train_df.values
x_test = test_df.values


In [ ]:
def show_imgs(n_rows,n_cols,x_test,y_label):
    assert len(x_test) == len(y_label)
    assert n_rows * n_cols < len(x_test)
    plt.figure(figsize = (n_cols * 2.8,n_rows * 3.2))
    for row in range(n_rows):
        for col in range(n_cols):
            index = n_cols * row + col
            plt.subplot(n_rows,n_cols,index+1)
            plt.imshow(x_test[index].reshape(28,28),cmap = "gray",interpolation = "nearest")
            plt.axis("off")
            plt.title(y_label[index])
    plt.show()

y_test_dict = dict(y_test)
y_test_label = []

for i in y_test_dict.keys():
    y_test_label.append(y_test_dict[i])

show_imgs(2,5,x_test,y_test_label)

In [ ]:
# 显示label在数据集中的位置
label_binarizer = LabelBinarizer()
y_train = label_binarizer.fit_transform(y_train)
y_test = label_binarizer.fit_transform(y_test)
# print(y_test[0])
# print(len(y_test[0]))

# 将数组中的元素转换为[0.1]区间内的数字
x_train = x_train / 255
x_test = x_test / 255

# 通过reshape数据将数据模型从1-D-->3-D来适应CNN
x_train = x_train.reshape(-1,28,28,1)
x_test = x_test.reshape(-1,28,28,1)

# print(x_train)
# print(len(x_test))
# print(len(y_train))
# print(len(y_test))

In [ ]:
# 对训练集进行预处理防止过拟合
datagen = ImageDataGenerator(
#   some parameter  
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=5,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False  # randomly flip images

)
datagen.fit(x_train)
# print(x_train[0][0])

In [ ]:
model = Sequential()

model.add(Conv2D(64, (3, 3), activation = 'relu',padding = 'same', input_shape=(28, 28, 1)))
model.add(BatchNormalization())
model.add(MaxPool2D(2, 2))
model.add(Conv2D(128, (3, 3),padding = 'same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(2, 2))
model.add(Conv2D(512, (3, 3),padding = 'same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(2, 2))
model.add(Conv2D(512, (3, 3),padding = 'same', activation='relu'))
model.add(Flatten())
model.add(Dense(512, activation='relu'))

model.add(Dense(units = 24 , activation = 'softmax'))
model.compile(optimizer = 'sgd' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])
model.summary()

In [ ]:
# print(type(x_train),type(x_test))
# print(type(y_train),type(y_test))
# print(x_train[0][0],x_test[0][0])
# print(y_train[0],y_test[0])

In [ ]:
times = 10 # 训练次数

# 保存最好的模型
checkpointer = ModelCheckpoint(filepath="gesture.model.weights.best.0703.h5", verbose = 1, save_best_only=True)

history = model.fit(datagen.flow(x_train,y_train, batch_size = 32),
                    epochs = times, 
                    validation_data = (x_test, y_test),
                    callbacks = [checkpointer])

In [ ]:
epochs = [i for i in range(times)]
fig , ax = plt.subplots(1,2)
train_acc = history.history['accuracy']
train_loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']
fig.set_size_inches(16,9)

ax[0].plot(epochs , train_acc , 'go-' , label = 'Training Accuracy')
ax[0].plot(epochs , val_acc , 'ro-' , label = 'Testing Accuracy')
ax[0].set_title('Training & Validation Accuracy')
ax[0].legend()
ax[0].set_xlabel("Epochs")
ax[0].set_ylabel("Accuracy")
ax[0].grid(True)
#ax[0].set_ylim(0.96,1)

ax[1].plot(epochs , train_loss , 'g-o' , label = 'Training Loss')
ax[1].plot(epochs , val_loss , 'r-o' , label = 'Testing Loss')
ax[1].set_title('Testing Accuracy & Loss')
ax[1].legend()
ax[1].set_xlabel("Epochs")
ax[1].set_ylabel("Loss")
ax[1].grid(True)

plt.show()

In [ ]:
from tensorflow.keras.models import load_model
load_model = load_model("gesture.model.weights.best.0703.h5")
# 用测试集评估模型准确度
score = load_model.evaluate(x_test, y_test,verbose = 1)

# 打印出准确度
print('\n', 'Test accuracy:', score[1])
print(score)

In [ ]:
# 从测试集中随机取出数据验证模型准确度

y_hat = load_model.predict(x_test)

#print(y_hat[0])
class_names = np.arange(0,24)
# print(class_names)

figure = plt.figure(figsize=(20, 8))
for i, index in enumerate(np.random.choice(x_test.shape[0], size=15, replace=False)):
    ax = figure.add_subplot(3, 5, i + 1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(x_test[index]),cmap = "gray")
    predict_index = np.argmax(y_hat[index])
    true_index = np.argmax(y_test[index])
    ax.set_title("{} ({})".format(class_names[predict_index], 
                                  class_names[true_index]),
                                  color=("green" if predict_index == true_index else "red"))